# Calculate trends by start year, end year, season, etc.

Calculates linear trends over regional averages of any rainfall files in the `dir_list['raw']` directory.

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import warnings
import os
import re
import glob
from tqdm.notebook import tqdm

from funcs_support import get_params, get_filepaths, utility_print, area_mean, subset_to_srat
from funcs_load import load_cam6,load_raws
from funcs_support2 import seasmeans
dir_list = get_params()

In [2]:
# Reference model for regridding
ref_mod = 'GPCP'
# Filename suffix for loading / saving
suffix = 'HoAfrica'
# Future experiments to extend historic runs on
future_exps = ['ssp245','ssp585'] #ssp585

# Range over which to calculate standard deviation
sd_range = slice(1981,2014)

overwrite = False

In [3]:
# The whole region
subset_params_hoa = {'lon':slice(32,55),'lat':slice(-3,12.5),'time':slice('1958-01-01','2023-12-30')}
#subset_params_hoa = {'lon':slice(32,55),'lat':slice(-3,12.5),'time':slice('1850-01-01','2023-12-30')}

In [4]:
dss = dict()

## Calculating trends

In [5]:
def calc_trends(dss,
                # Get seasonal means
                seasons = {'MAM':[3,5],
                           'AMJ':[4,6],
                           'MJJ':[5,7],
                           'SON':[9,11],
                           'OND':[10,12]},
                # Regional subsets 
                # (all will be pushed through `subset_to_srat()`) 
                # or None, for by-pixel (after regridding) processing
                regs = {'HoA-bimod':{'lon':slice(32,55),
                                     'lat':slice(-3,12.5)},
                        'overlap':{'lon':slice(38.5,47.5),
                                 'lat':slice(2,6.5)}},
                # To regrid to 
                ref_mod = ref_mod,
                # File suffixes for loading
                suffix = suffix,
                # Years over which to calculate variance for CV trends
                sd_range = sd_range,
                overwrite=False):

    # Add filename modifier if processing by pixel 
    if regs is None:
        suffix = suffix+'-bypixel'

    # Add filename modifier if not regridding
    if ref_mod is None:
        suffix = suffix+'-oggrid'

    for exp in tqdm([ex for ex in dss]):
        for mod in tqdm(dss[exp]):
            #----------- File Setup -----------
            if not os.path.exists(dir_list['proc']+mod):
                os.mkdir(dir_list['proc']+mod+'/')
                print(dir_list['proc']+mod+'/ created!')
    
            # Get spanned timeframe for output filename
            datestr = datestr = (re.sub(r'-','',str(dss[exp][mod].time[0].values)[0:7]+'01') + '-' +
                                 re.sub(r'-','',str(dss[exp][mod].time[-1].values)[0:7]+str(dss[exp][mod].time[-1].dt.daysinmonth.values)))
            # Get output filename
            output_fn = dir_list['proc']+mod+'/prtrends_Amon_'+mod+'_'+exp+'_allruns_'+datestr+'_'+suffix+'.nc'
    
            #----------- Processing -----------
            if (overwrite) or (not os.path.exists(output_fn)):
                
                #----- Load
                ds_tmp = dss[exp][mod].load()
                
                # Fixes issue with unsorted time dimension
                try: 
                    xr.testing.assert_allclose(ds_tmp.sortby('time').time,
                               ds_tmp.time)
                except:
                    print('had to sort time dimension of model '+mod+', exp '+exp)
                    ds_tmp = ds_tmp.sortby('time')
                    
                #----- Drop duplicate runs... 
                if 'run' in ds_tmp.sizes:
                    ds_tmp = ds_tmp.drop_duplicates(dim='run')
                
                #----- Get seasonal subsets 
                with warnings.catch_warnings(): 
                    # Ignore FutureWarning on time.dt calls
                    warnings.simplefilter(action='ignore', category=FutureWarning)
                    
                    ds_tmp = seasmeans(ds_tmp,seasons=seasons)
                
                #----- Regrid to reference
                if ref_mod is not None:
                    try:
                        ds_ref.load()
                    except: 
                        raise NameError('No `ds_ref` found; load the obs cell to get it as regridding target.')
                
                    try:
                        rgrd = xe.Regridder(ds_tmp,ds_ref,'conservative',ignore_degenerate=True)
                    except: 
                        print('regridding issue with exp '+exp+', model '+mod+' (possibly a non-rectangular grid?)')
                        continue
                    ds_tmp = rgrd(ds_tmp)
                    # Turn pixels that are completely 0 from regridding into nans 
                    ds_tmp = ds_tmp.where(ds_tmp.mean('year')!=0)
                
                #----- Get regional subsets and area mean
                if regs is not None:
                    ds_tmp = xr.concat([area_mean(subset_to_srat(ds_tmp.sel(**sp)))
                                        for reg,sp in regs.items()],
                                       dim = pd.Index([reg for reg in regs],name='region'))
                
                # Make sure partial seasons are nan (resample(time='1Y').mean(skipna=False 
                # should be doing this, but it's not.)
                for seas in seasons:
                    if dss[exp][mod].time[-1].dt.month<seasons[seas][1]:
                        ds_tmp['pr'].loc[{'season':seas,'year':ds_tmp.year.max()}] = np.nan
                
                #----- Calculate trends
                #-- Calculate OLS trends
                with warnings.catch_warnings(): 
                    # Ignore Rankwarning on polyfit calls
                    warnings.simplefilter('ignore')
                
                    # Calculate linear fits across years, through every possible 
                    # length of trend for each start year
                    fits = xr.concat([xr.concat([(ds_tmp.sel(year=slice(start_year,start_year+run_length)).
                                                               polyfit(dim='year',deg=1,skipna=False).
                                                               isel(degree=0,drop=True).
                                                               rename({'pr_polyfit_coefficients':'prtrend'}))
                                                           for run_length in np.arange(1,len(ds_tmp.year.sel(year=slice(start_year,None))))],
                                                          dim=pd.Index(start_year + np.arange(1,len(ds_tmp.year.sel(year=slice(start_year,None)))),name='end_year'))
                                                for start_year in ds_tmp.year[0:-1].values],
                                               dim=pd.Index(ds_tmp.year[0:-1].values,name='start_year'))
                
                #-- Calculate Theil-Sen slope estimator
                # Calculate slopes through each combination of two points
                point_slopes = xr.concat([xr.concat([(ds_tmp.sel(year=end_year)-ds_tmp.sel(year=start_year))/(end_year - start_year)
                                                      if end_year > start_year else ds_tmp.sel(year=end_year)*np.nan
                                                     for start_year in ds_tmp.year[0:-1].values],
                                                    dim=pd.Index(ds_tmp.year[0:-1].values,name='start_year')).drop_vars('year',errors='ignore')
                                          for end_year in ds_tmp.year[1:None].values],
                                         dim=pd.Index(ds_tmp.year[1:None].values,name='end_year'))
                
                # Get median across combinations of points to get Thiel-Sen
                # Estimator
                fits['pr_tsslope'] = xr.concat([xr.concat([(point_slopes.sel(start_year = slice(start_year,end_year-1),
                                                                 end_year = slice(start_year+1,end_year)).
                                                      median(('end_year','start_year')))
                                                         for start_year in ds_tmp.year[0:-1].values],
                                                        dim=pd.Index(ds_tmp.year[0:-1].values,name='start_year')).drop_vars('year',errors='ignore')
                                         for end_year in ds_tmp.year[1:None].values],
                                        dim=pd.Index(ds_tmp.year[1:None].values,name='end_year'))['pr']

                # Make sure to mask out trends that should be nan because they
                # have missing data (can't use skipna=False in the median above, since
                # start_year>=end_year are nans)
                fits['pr_tsslope'] = fits['pr_tsslope'].where(~np.isnan(point_slopes.pr))
                                                      
                # Add run length variable
                trend_length = fits.end_year-fits.start_year
                trend_length = trend_length.where(trend_length>0)
                fits['trend_length'] = trend_length
                
                #----- Calculate IA SDs
                fits['pr_std'] = ds_tmp['pr'].sel(year=sd_range).std(dim='year')
                fits['pr_std'].attrs['Range'] = 'Calculated on '+str(sd_range.start)+'-'+str(sd_range.stop)
                fits['pr_std'].attrs['long_name'] = r'$\sigma{P}$ ('+ str(sd_range.start)+'-'+str(sd_range.stop)+')'
                
                #----- Calculate IA IQRs
                fits['pr_iqr'] = (ds_tmp['pr'].sel(year=sd_range).quantile(0.75,dim='year') - 
                                  ds_tmp['pr'].sel(year=sd_range).quantile(0.25,dim='year'))
                fits['pr_std'].attrs['Range'] = 'Calculated on '+str(sd_range.start)+'-'+str(sd_range.stop)
                fits['pr_std'].attrs['long_name'] = r'0.75 - 0.25 IQR of $P$ ('+ str(sd_range.start)+'-'+str(sd_range.stop)+')'
                
                
                #----- Annotate
                fits['prtrend'].attrs['Units'] = 'mm/day/year'
                fits['trend_length'].attrs['Units'] = 'years'
                fits['prtrend'].attrs['long_name'] = r'$P$ trend'
                
                fits.attrs['SOURCE'] = 'calculate_trends_variablelength.ipynb'
                if regs is None:
                    fits.attrs['DESCRIPTION'] = 'linear trends (slope), calculated through every possible every start/end year combination'
                else:
                    fits.attrs['DESCRIPTION'] = 'linear trends (slope), calculated through every possible every start/end year combination'
                if ref_mod is not None:
                    fits.attrs['REGRIDDING'] = 'conservatively regridded to '+ref_mod+' before processing'
    
    
                #----- Save
                if os.path.exists(output_fn):
                    os.remove(output_fn)
                    print(output_fn+' removed to allow overwrite!')
    
                fits.to_netcdf(output_fn)
                print(output_fn+' saved!')
    
            else:
                print(output_fn+' already exists, skipped!')

## Gridded obs data

In [6]:
# Directly load, since for some reason load_raws is being slow here 
dss['obs'] = dict()
for row in get_filepaths().query('varname=="pr" and freq=="Amon" and exp=="historical" and run=="obs"').iterrows():
    dss['obs'][row[1].model] = xr.open_dataset(row[1].path)
    dss['obs'][row[1].model] = xa.fix_ds(dss['obs'][row[1].model]).sel(**subset_params_hoa)

In [7]:
# Get reference model that others are regridded to
ds_ref = dss['obs'][ref_mod]

In [8]:
# Calculate trends
calc_trends(dss)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

/dx02/kschwarz/climate_proc/UDel/prtrends_Amon_UDel_obs_allruns_19000101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CHIRPSv3/prtrends_Amon_CHIRPSv3_obs_allruns_19810101-20230630_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CHIRPS/prtrends_Amon_CHIRPS_obs_allruns_19810101-20220930_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/GPCP/prtrends_Amon_GPCP_obs_allruns_19790101-20230430_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/GPCC-monitoring/prtrends_Amon_GPCC-monitoring_obs_allruns_19820101-20231031_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/GPCC-full/prtrends_Amon_GPCC-full_obs_allruns_18910101-20201231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CMORPH/prtrends_Amon_CMORPH_obs_allruns_19980101-20230430_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CMAP/prtrends_Amon_CMAP_obs_allruns_19790101-20231231_HoAfrica.nc already exists, skipped!


In [ ]:
# Calculate trends by pixel, original grids of each obs product 
calc_trends(dss,overwrite=False,ref_mod = None,regs = None)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/UDel/prtrends_Amon_UDel_obs_allruns_19580101-20141231_HoAfrica-bypixel-oggrid.nc saved!


In [9]:
del dss['obs']

## Station obs data

In [11]:
# Directly load, since for some reason load_raws is being slow here 
dss['stations'] = dict()
for row in get_filepaths().query('varname=="pr" and freq=="Amon" and exp=="historical" and suffix == "HoAfrica-stns"').iterrows():
    dss['stations'][row[1].model] = xr.open_dataset(row[1].path)

In [ ]:
calc_trends(dss,regs=None,ref_mod = None)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/GHCN/prtrends_Amon_GHCN_stations_allruns_18780401-20240531_HoAfrica-bypixel-oggrid.nc saved!


In [18]:
del dss['stations']

## AMIP

In [30]:
dss['amip'] = load_raws({'varname':'pr','freq':'Amon','suffix':suffix,'exp':'amip'},
                        treat_multiple={'time':'earliest'},
                        force_load = True)

In [31]:
# Convert from kg/m^2s to mm/day
dss['amip'] = {mod:ds*60*60*24 for mod,ds in dss['amip'].items()}

In [32]:
# CIESM is in m/s for some reason, so the above calc is off by 1000
dss['amip']['CIESM']['pr'] = dss['amip']['CIESM']['pr']*1000

In [13]:
# Calculate trends
calc_trends({'amip':dss['amip']})

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

/dx02/kschwarz/climate_proc/ACCESS-ESM1-5/prtrends_Amon_ACCESS-ESM1-5_amip_allruns_19790101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/BCC-CSM2-MR/prtrends_Amon_BCC-CSM2-MR_amip_allruns_19790101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/BCC-ESM1/prtrends_Amon_BCC-ESM1_amip_allruns_19790101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CAMS-CSM1-0/prtrends_Amon_CAMS-CSM1-0_amip_allruns_19790101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CAS-ESM2-0/prtrends_Amon_CAS-ESM2-0_amip_allruns_19780101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CESM2/prtrends_Amon_CESM2_amip_allruns_19500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CESM2-FV2/prtrends_Amon_CESM2-FV2_amip_allruns_19500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CESM2-WACCM/prtrends_Amon_CESM2-WACCM_amip_allruns_195

In [ ]:
# Calculate trends by pixel
calc_trends({'amip':dss['amip']},
            regs = None,
            overwrite=True)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/ACCESS-ESM1-5/prtrends_Amon_ACCESS-ESM1-5_amip_allruns_19790101-20141231_HoAfrica-bypixel.nc removed to allow overwrite!
/dx02/kschwarz/climate_proc/ACCESS-ESM1-5/prtrends_Amon_ACCESS-ESM1-5_amip_allruns_19790101-20141231_HoAfrica-bypixel.nc saved!


/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


...

/dx02/kschwarz/climate_proc/CESM2-WACCM-FV2/prtrends_Amon_CESM2-WACCM-FV2_amip_allruns_19500101-20141231_HoAfrica-bypixel.nc saved!


/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/CIESM/prtrends_Amon_CIESM_amip_allruns_19790101-20141231_HoAfrica-bypixel.nc removed to allow overwrite!
/dx02/kschwarz/climate_proc/CIESM/prtrends_Amon_CIESM_amip_allruns_19790101-20141231_HoAfrica-bypixel.nc saved!


/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/CanESM5/prtrends_Amon_CanESM5_amip_allruns_19500101-20141231_HoAfrica-bypixel.nc saved!


In [36]:
del dss['amip']

## CMIP6 / coupled data

In [77]:
# Load historical and future CMIP6 coupled data
dss['coupled'] = {exp:load_raws({'varname':'pr','freq':'Amon','suffix':suffix,'exp':exp},
                                treat_multiple={'time':'earliest'},
                                subset_params = {'time':slice('1958-01-01','2100-12-30')},#{'time':slice('1850-01-01','2023-12-30')},
                                force_load = True) #
                  for exp in ['historical',*future_exps]}

/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/xarray/coding/times.py:995: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/xarray/core/indexing.py:630: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  array = array.get_duck_array()
/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/xarray/core/indexing.py:500: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)
/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-

In [78]:
# Convert from kg/m^2s to mm/day
dss['coupled'] = {exp:{mod:ds*60*60*24 for mod,ds in dss['coupled'][exp].items()}
                  for exp in dss['coupled']}

In [79]:
### Stitch together historical and future runs to run until 2023
# Get which models' data can be extended from historical 
# to ssp245
extend_mods = {future_exp:[mod for mod in dss['coupled']['historical']
                               if mod in dss['coupled'][future_exp]]
                               for future_exp in future_exps}

# First, the historical and future need to be put onto the same calendar;
# otherwise the julian / gregorian / noleap issue runs up
# (this may cause slight issues with weigthing of months, but it should be
# minor...)
for exp in dss['coupled']:
    for mod in dss['coupled'][exp]:
        dss['coupled'][exp][mod]['time'] = pd.date_range(str(dss['coupled'][exp][mod].time.min().values)[0:7]+'-01',
                                                         str(dss['coupled'][exp][mod].time.max().values)[0:7]+'-28',
                                                          freq='1MS') + pd.Timedelta(days=15)

# Now, stitch together historical and future runs where possible
for future_exp in extend_mods:
    dss['coupled']['hist-'+future_exp] = dict()

    for mod in extend_mods[future_exp]:
        dss['coupled']['hist-'+future_exp][mod] = xr.concat([dss['coupled']['historical'][mod],
                                                        dss['coupled'][future_exp][mod].drop_duplicates('run')],
                                                       dim='time').drop_duplicates('time')

# Now have hist be just the ones left that aren't extended at all 
# (to not double-do it too much...)
dss['coupled']['hist-none'] = {mod:ds for mod,ds in dss['coupled']['historical'].items()
                          if mod not in np.unique(np.hstack([mods for exp,mods in extend_mods.items()]))}

# Remove unnecessary copies
for exp in [exp for exp in dss['coupled'] if re.search(r'hist-',exp)]:
    dss[exp] = dss['coupled'][exp]
del dss['coupled']

In [47]:
seasons = {'MAM':[3,5],
           'AMJ':[4,6],
           'MJJ':[5,7],
           'SON':[9,11],
           'OND':[10,12]}
# Regional subsets 
# (all will be pushed through `subset_to_srat()`) 
regs = {'HoA-bimod':{'lon':slice(32,55),
                     'lat':slice(-3,12.5)},
        'overlap':{'lon':slice(38.5,47.5),
                 'lat':slice(2,6.5)}}

In [19]:
# Calculate trends
calc_trends(dss)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

/dx02/kschwarz/climate_proc/ACCESS-CM2/prtrends_Amon_ACCESS-CM2_hist-ssp245_allruns_18500101-20231231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/ACCESS-ESM1-5/prtrends_Amon_ACCESS-ESM1-5_hist-ssp245_allruns_18500101-20231231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CAS-ESM2-0/prtrends_Amon_CAS-ESM2-0_hist-ssp245_allruns_18500101-20231231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CESM2/prtrends_Amon_CESM2_hist-ssp245_allruns_18500101-20231231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CESM2-WACCM/prtrends_Amon_CESM2-WACCM_hist-ssp245_allruns_18500101-20231231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CNRM-CM6-1/prtrends_Amon_CNRM-CM6-1_hist-ssp245_allruns_18500101-20231231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CNRM-ESM2-1/prtrends_Amon_CNRM-ESM2-1_hist-ssp245_allruns_18500101-20231231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc

  0%|          | 0/19 [00:00<?, ?it/s]

/dx02/kschwarz/climate_proc/AWI-CM-1-1-MR/prtrends_Amon_AWI-CM-1-1-MR_hist-none_allruns_18500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/BCC-CSM2-MR/prtrends_Amon_BCC-CSM2-MR_hist-none_allruns_18500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/BCC-ESM1/prtrends_Amon_BCC-ESM1_hist-none_allruns_18500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CAMS-CSM1-0/prtrends_Amon_CAMS-CSM1-0_hist-none_allruns_18500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CESM2-FV2/prtrends_Amon_CESM2-FV2_hist-none_allruns_18500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CESM2-WACCM-FV2/prtrends_Amon_CESM2-WACCM-FV2_hist-none_allruns_18500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CMCC-CM2-SR5/prtrends_Amon_CMCC-CM2-SR5_hist-none_allruns_18500101-20141231_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_

In [81]:
for exp in ['hist-none','hist-ssp245','hist-ssp585']:
    for mod in dss[exp]:
        dss[exp][mod] = dss[exp][mod].drop_duplicates('time').sel(time=slice('1958-01-01','2024-12-30'))

In [86]:
# Calculate trends by pixel
calc_trends({exp:dss[exp] for exp in ['hist-ssp245','hist-ssp585','hist-none']},
            regs = None,
            overwrite=True)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]

/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/ACCESS-CM2/prtrends_Amon_ACCESS-CM2_hist-ssp245_allruns_19580101-20241231_HoAfrica-bypixel.nc saved!


/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


...

/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/MPI-ESM-1-2-HAM/prtrends_Amon_MPI-ESM-1-2-HAM_hist-none_allruns_19580101-20141231_HoAfrica-bypixel.nc removed to allow overwrite!
/dx02/kschwarz/climate_proc/MPI-ESM-1-2-HAM/prtrends_Amon_MPI-ESM-1-2-HAM_hist-none_allruns_19580101-20141231_HoAfrica-bypixel.nc saved!


/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


/dx02/kschwarz/climate_proc/NorCPM1/prtrends_Amon_NorCPM1_hist-none_allruns_19580101-20141231_HoAfrica-bypixel.nc removed to allow overwrite!
/dx02/kschwarz/climate_proc/NorCPM1/prtrends_Amon_NorCPM1_hist-none_allruns_19580101-20141231_HoAfrica-bypixel.nc saved!
/dx02/kschwarz/climate_proc/TaiESM1/prtrends_Amon_TaiESM1_hist-none_allruns_19580101-20141231_HoAfrica-bypixel.nc saved!


/home/kschwarz/miniforge3/envs/climate/lib/python3.12/site-packages/numpy/lib/nanfunctions.py:1563: RuntimeWarning: All-NaN slice encountered
  return function_base._ureduce(a,


In [87]:
for exp in ['hist-ssp245','hist-ssp585','hist-none']:
    del dss[exp]

## NMME hindcasts

In [20]:
dss['hindcastsf'] = load_raws({'varname':'pr','freq':'Amon','suffix':suffix,'exp':'hindcastsf','run':'NMME'},
                              force_load = True)

In [21]:
# Calculate trends
calc_trends(dss)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

/dx02/kschwarz/climate_proc/CMC1-CanCM3/prtrends_Amon_CMC1-CanCM3_hindcastsf_allruns_19810101-20200630_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CMC2-CanCM4/prtrends_Amon_CMC2-CanCM4_hindcastsf_allruns_19810101-20200630_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CanCM4i/prtrends_Amon_CanCM4i_hindcastsf_allruns_19810101-20221031_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CanSIPS-IC3/prtrends_Amon_CanSIPS-IC3_hindcastsf_allruns_19800101-20240731_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/CanSIPSv2/prtrends_Amon_CanSIPSv2_hindcastsf_allruns_19810101-20221031_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/GEM-NEMO/prtrends_Amon_GEM-NEMO_hindcastsf_allruns_19810101-20221031_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/GFDL-SPEAR/prtrends_Amon_GFDL-SPEAR_hindcastsf_allruns_19910101-20240731_HoAfrica.nc already exists, skipped!
/dx02/kschwarz/climate_proc/NASA-GEOSS2S

In [22]:
del dss['hindcastsf']